In [1]:
!pip3 install pandas
!pip3 install sqlalchemy
!pip3 install psycopg2  # For PostgreSQL
!pip3 install pymysql   # For MySQL
!pip3 install cx_Oracle # For Oracle 
!pip3 install pyodbc


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


Setup and Imports


In [2]:
# Import necessary libraries
import pandas as pd
from sqlalchemy import create_engine, inspect
import pyodbc

# Helper function to normalize names
def normalize_names(name):
    """Normalize table and column names for comparison."""
    return name.lower().replace('_', '')

# Example function to connect to a database (replace with actual credentials)
def connect_to_database(server, database, username, password):
    # Create a connection string using pyodbc
    connection_string = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"
    
    # Create a SQLAlchemy engine using the connection string
    engine = create_engine(f"mssql+pyodbc:///?odbc_connect={connection_string}")
    
    # Return the engine connection
    return engine.connect()

# Example usage:
server = "MSSQLSERVER01"
database = "ORACLE_EBS_HACK"
username = "Sanap2"
password = "Rahul123@"

connection = connect_to_database(server, database, username, password)

# Now you can use the connection to query the database
# For example:
df = pd.read_sql_query("SELECT * FROM your_table_name", connection)
print(df)

ImportError: libodbc.so.2: cannot open shared object file: No such file or directory

Schema Extraction


In [ ]:
def extract_schema(db_connection):
    """Extracts schema information (tables, columns, data types) from a database."""
    schema = {}
    inspector = inspect(db_connection)

    for table_name in inspector.get_table_names():
        columns = inspector.get_columns(table_name)
        schema[table_name] = [{'name': col['name'], 'data_type': col['type'], 'length': col.get('length')} for col in columns]
    
    return schema

# Example usage (replace with actual database connections)
# source_db_conn = connect_to_database("source_db_connection_string")
# target_db_conn = connect_to_database("target_db_connection_string")
# source_schema = extract_schema(source_db_conn)
# target_schema = extract_schema(target_db_conn)

Shema Comparison

In [ ]:
def compare_schemas(source_schema, target_schema):
    """Compares the source and target schemas and identifies potential mappings."""
    mappings = []
    for source_table, source_columns in source_schema.items():
        for target_table, target_columns in target_schema.items():
            if tables_are_similar(source_table, target_table):
                for source_column in source_columns:
                    for target_column in target_columns:
                        if columns_are_similar(source_column, target_column):
                            mapping_type = determine_mapping_type(source_column, target_column)
                            mappings.append({
                                'source_table': source_table, 
                                'source_column': source_column['name'],
                                'target_table': target_table, 
                                'target_column': target_column['name'],
                                'mapping_type': mapping_type
                            })
    return mappings

def tables_are_similar(source_table, target_table):
    """Determines if two tables are similar based on normalized names."""
    return normalize_names(source_table) == normalize_names(target_table)

def columns_are_similar(source_column, target_column):
    """Determines if two columns are similar based on name, data type, and length."""
    return (normalize_names(source_column['name']) == normalize_names(target_column['name']) and
            source_column['data_type'] == target_column['data_type'] and
            (source_column['length'] == target_column['length'] or 
             source_column['length'] is None or 
             target_column['length'] is None))

def determine_mapping_type(source_column, target_column):
    """Determines the type of mapping (EQ or SQL) based on column data types."""
    if source_column['data_type'] == target_column['data_type']:
        return "EQ"
    else:
        return "SQL"

# Example comparison
# mappings = compare_schemas(source_schema, target_schema)
# print(mappings)

Value Mapping

In [ ]:
def generate_sql_mapping(source_column, target_column):
    """Generates SQL transformation logic if needed."""
    if source_column['data_type'] == 'DATE' and target_column['data_type'] == 'VARCHAR':
        return f"TO_CHAR({source_column['name']}, 'YYYY-MM-DD')"
    # Add more transformation rules as necessary
    return None

Output Generation

In [ ]:
def generate_mapping_report(mappings):
    """Generates a mapping report as a DataFrame."""
    df = pd.DataFrame(mappings)
    return df

# Example output
# report = generate_mapping_report(mappings)
# report.to_csv('mapping_report.csv', index=False)
# print(report)

Combine all Pieces

In [ ]:
def main(source_db_conn, target_db_conn):
    # Extract schemas
    source_schema = extract_schema(source_db_conn)
    target_schema = extract_schema(target_db_conn)
    
    # Compare schemas and generate mappings
    mappings = compare_schemas(source_schema, target_schema)
    
    # Generate and output the mapping report
    report = generate_mapping_report(mappings)
    
    # Save the report to a file or display it
    report.to_csv('mapping_report.csv', index=False)
    print(report)

# Example execution
# source_db_conn = connect_to_database("source_db_connection_string")
# target_db_conn = connect_to_database("target_db_connection_string")
# main(source_db_conn, target_db_conn)